# WB-NLP-2022 PD1 - Preprocessing przykładowych danych, eksploracja
**Mateusz Krzyziński, Piotr Wilczyński, Artur Żółkowski**

Poniższy notebook jest rozwiązaniem pracy domowej numer 1 i dotyczy eksploracji zbioru Database of Parliamentary Speeches in Ireland, 1919-2013, zawierającego teksty przemówień parlamentarzystów irlandzkich. 

#### Ładowanie danych i pakietów

In [ ]:
! pip install swifter
! pip install pandas
! pip install textacy
! pip install -U kaleido

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import pandas as pd
from tqdm.auto import tqdm 
import swifter
import plotly.express as px
from wordcloud import WordCloud
from matplotlib import pyplot as plt
import numpy as np
import textacy
from IPython.display import Image
pd.options.plotting.backend = "plotly"

In [ ]:
!wget -O data.tar.gz "https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/6MZN76/CRUNF0"
!wget -O Dail_debates_1937-2011_ministers.tab "https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/6MZN76/BFFQKZ"

In [ ]:
!tar -xf data.tar.gz

In [ ]:
en = spacy.load("en_core_web_sm")
df = pd.read_table('Dail_debates_1919-2013.tab')

In [ ]:
df.head(5)

### Wstępne informacje

Cała ramka danych zawiera 4 443 713 wierszy (przemówień) i 10 kolumn, w tym 6 kluczowych z punktu widzenia eksploracji danych:
- tytuł przemówienia,
- data przemówienia,
- imię i nazwisko przemawiającego,
- nazwa partii politycznej,
- nazwa okręgu wyborczego, z którego był wybrany parlamentarzysta,
- tekst przemówienia.

Nie ma żadnych braków danych. 

In [ ]:
df.isnull().sum()

In [ ]:
df.date = pd.to_datetime(df.date)

### Krótka eksploracja danych pozatekstowych 
(niezbędne dla kontekstu dalszej eksploracji) 

Dane są za okres od 1919 do 2013 roku. W tym czasie swoje wystąpienia w parlamencie miało 1178 parlamentarzystów z 27 różnych partii. 

W irlandzkim parlamencie (dokładniej niższej jego izbie - Dáil Éireann) zasiada 160 parlamentarzystów. 

W zbiorze danych znajdują się przemówienia parlamentarzystów wybranych z 151 okręgów wyborczych (natomiat okręgi zmieniały się w czasie - obecnie jest ich 39). 

In [ ]:
df.date.min(), df.date.max()

In [ ]:
df.constID.nunique(), df.partyID.nunique(), df.memberID.nunique()

Wyraźnie widać, że najwięcej przemówień wygłosili przedstawiciele czterech partii (które miały też najwięcej parlamentarzystów w izbie w tym okresie). Warto przyjrzeć się krótko tym partiom. 

- **Fianna Fail - The Republican Party** (*Żołnierze Losu*) - założona w 1926, partia konserwatywna, chrześcijańsko-demokratyczna, wielokrotnie rządziła sama lub w koalicji, sprzeciwia się silnej integracji europejskiej 
- **Fine Gael** (*Rodzina Irlandczyków*) - założona w 1933, liberalno-konserwatywa, chrześcijańsko-demokratyczna, uważana za skłonną do kompromisów z Wielką Brytanią, zwolenniczka integracji europejskiej
- **The Labour Party** - założona w 1912, partia centro-lewicowa, socjaldemokratyczna, proeuropejska, trzecia główna partia, rządząca w koalicjach z Fine Gael 
- **Sinn Féin** (*My Sami*) - założona w 1905 (w obecnej formie w 1970), partia lewicowo-nacjonalistyczna, republikańska,  jej głównym celem pozostaje zjednoczenie obu części Irlandii w jedną republikę

In [ ]:
fig = df.party_name.value_counts().plot(kind='bar')
img_bytes = fig.to_image(format="png", width=800, height=400, scale=2, engine='kaleido')

In [ ]:
#w notatniku
fig
#do pdf 
Image(img_bytes)

In [ ]:
fig = df.groupby("party_name").member_name.nunique().sort_values(ascending=False).plot(kind='bar')
img_bytes = fig.to_image(format="png", width=800, height=400, scale=2, engine='kaleido')
Image(img_bytes)

Dla zmniejszenia rozmiaru danych ograniczamy się do przemów w parlamencie począwszy od 6 czerwca 2002 roku, kiedy miało miejsce zaprzysiężenie parlamentarzystów 29-tego Dáil Éireann (Zgromadzenia). Dane z tego okresu stanowią ponad 25% wszystkich danych. 

Ponadto dla tego okresu podział na partie wygląda podobnie, choć przewaga dwóch największych partii jest bardziej widoczna, a Sinn Fein straciło popularność wśród wyborców. 

In [ ]:
df2 = df[df.date >= np.datetime64('2002-06-06')]

In [ ]:
len(df2)/len(df)

In [ ]:
fig = df2.party_name.value_counts().plot(kind='bar')
img_bytes = fig.to_image(format="png", width=800, height=400, scale=2, engine='kaleido')
Image(img_bytes)

In [ ]:
fig = df2.groupby("party_name").member_name.nunique().sort_values(ascending=False).plot(kind='bar')
img_bytes = fig.to_image(format="png", width=800, height=400, scale=2, engine='kaleido')
Image(img_bytes)

In [ ]:
df2.constID.nunique(), df2.partyID.nunique(), df2.memberID.nunique()

Ponadto z wybranego okresu bierzemy losowy sample przemówień. Ostatecznie będziemy pracować na około 23 tys. przemówień. 

In [ ]:
df3 = df2.sample(frac=.02, random_state=42)
len(df3)

## Analiza ze względu na długość przemówień

In [ ]:
tqdm.pandas()
docs = df3['speech'].swifter.apply(en)

Rozkład długości wystąpień - najwięcej jest krótkich przemówień, do 1000 słów. Te powyżej 2000, a w szczególności 3000 tysięcy sprawiają wrażenie outlierów. Spróbujemy się coś o nich dowiedzieć. 

In [ ]:
doc_lens = docs.str.len()
fig = doc_lens.hist(log_y=True) 
img_bytes = fig.to_image(format="png", width=800, height=400, scale=2, engine='kaleido')
Image(img_bytes)

In [ ]:
ministers_df = pd.read_table("Dail_debates_1937-2011_ministers.tab")
ministers_df = ministers_df[~ministers_df.memberID.isna()]
ministers_df["memberID"] = ministers_df["memberID"].astype(int)

Najdłuższa z wypowiedzi dotyczyła ustawy o napojach alkoholowych. Wygłosił ją w czerwcu 2003 roku Michael McDowell, który w ówczesnym rządzie pełnił funkcję Ministra Sprawiedliwości, Równości i Reform Prawnych. Widzimy również, że część z najdłuższych wystąpień jest powiązana ze sprawami budżetu, czysto finansowymi. Warto sprawdzić, jak często to właśnie ministrowie są "odpowiedzialni" za najdłuższe przemówienia.

In [ ]:
df3.title[doc_lens[doc_lens > 3000].sort_values(ascending=False).index]

In [ ]:
df3.loc[3378357]

In [ ]:
ministers_df[ministers_df.memberID == 719]

In [ ]:
long_speeches_idx = doc_lens[doc_lens >= 2000].sort_values(ascending=False).index
short_speeches_idx = doc_lens[doc_lens < 2000].sort_values(ascending=False).index

In [ ]:
ministers_ids = ministers_df.memberID.unique()

In [ ]:
ministers_ratio_long = np.mean(df3.loc[long_speeches_idx].memberID.isin(ministers_ids))
ministers_ratio_short = np.mean(df3.loc[short_speeches_idx].memberID.isin(ministers_ids))
ministers_ratio = np.mean(df3.memberID.isin(ministers_ids))

ministers_ratio, ministers_ratio_short, ministers_ratio_long

Ta hipoteza się nie potwierdziła - nie tylko przemówienia ministrów są długie (dłuższe niż 2000 słów) szczególnie często. Jest wręcz odwrotnie - ich procentowy udział w dłuższych przemówieniach jest mniejszy aniżeli w ogóle i w krótszych. Natomiast widać wyraźną różnice pomiędzy poszczególnymi politykami, jeżeli chodzi o ich średnią długość, jak i ilość wystąpień. 

In [ ]:
df3["speech_length"] = doc_lens

In [ ]:
speakers = df3.groupby(["memberID", "member_name", "party_name"]).agg({'speech_length': ['mean', 'size']}).reset_index()

In [ ]:
longest_speaches = speakers.sort_values([('speech_length', 'mean')], ascending=False)
longest_speaches

In [ ]:
most_speaches = speakers.sort_values([('speech_length', 'size')], ascending=False)
most_speaches

Spośród 25 najczęściej wypowiadających się polityków tylko jeden nie był ministrem. Jest to Caoimhghín Ó Caoláin - polityk Sinn Féin obecny w każdej z analizowanych kadencji.  

In [ ]:
most_speaches.memberID[:25].isin(ministers_ids)

In [ ]:
most_speaches.loc[85]

Wyraźne są również dość wyraźne różnice pomiędzy długościami wypowiedzi przedstawicieli poszczególnych partii. 

In [ ]:
fig = df3.groupby(["party_name"]).agg({'speech_length': 'mean'}).sort_values("speech_length", ascending=False).plot(kind='bar')
img_bytes = fig.to_image(format="png", width=800, height=400, scale=2, engine='kaleido')
Image(img_bytes)

In [ ]:
df3.party_name.value_counts()

## Analiza ze względu na treść przemówień

In [ ]:
lemmas = docs.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct])

In [ ]:
from collections import Counter
word_counts = Counter(lemmas.sum())

W chmurze słów dla wszystkich analizowanych przemówień pojawiają się słowa związane z politycznym dyskursem, np. powiązane z pozycjami/strukturami: Minister, Department, Government, Deputy, House. 

In [ ]:
wc = WordCloud(width=800, height=600)
wc.generate_from_frequencies(frequencies=word_counts)
plt.figure(figsize=(16,12))
plt.imshow(wc)
plt.show()

Dokładniejszy insight w najczęściej używane słowa daje wykres słupkowy. Tutaj możemy dostrzec kilka ciekawych rzeczy, niezwiązanych już tylko z samym dyskursem. 

- Wśród najpopularniejszych słów są np. 'school' i 'child', co może wskazywać na to, że często poruszany był temat edukacji. 
- Dość zaskakujące jest też częstotliwość występowania '€' - z pozoru mogłoby się wydawać, że politycy rzadko w wystąpieniach operują konkretnymi kwotami pieniężnymi.   
- Ciekawym jest też częste występowanie słowa 'person' będącego bardziej osobistym niż grupowe 'people'. Wartym uwagi pytaniem badawczym byłaby analiza, jak wyglądają zależności względem częstości użycia tych właśnie słów.  


In [ ]:
counts = pd.DataFrame(word_counts.most_common(30), columns=['word', 'count'])

In [ ]:
fig = px.bar(counts,orientation='h', y='word', x='count')
fig['layout']['yaxis']['autorange'] = "reversed"
fig.update_layout(bargap=0.2, font={'size':10})
img_bytes = fig.to_image(format="png", width=800, height=600, scale=2, engine='kaleido')
Image(img_bytes)

In [ ]:
party_names = ["Fianna Fáil", "Fine Gael", "The Labour Party", "Sinn Féin"]     
FF_indices = df3.index[df3.party_name == party_names[0]]
FG_indices = df3.index[df3.party_name == party_names[1]]
LP_indices = df3.index[df3.party_name == party_names[2]]
SF_indices = df3.index[df3.party_name == party_names[3]]

In [ ]:
FF_word_counts = Counter(lemmas[FF_indices].sum())
FG_word_counts = Counter(lemmas[FG_indices].sum())

In [ ]:
LP_word_counts = Counter(lemmas[LP_indices].sum())
SF_word_counts = Counter(lemmas[SF_indices].sum())

In [ ]:
FF_top_100_words = pd.DataFrame(FF_word_counts.most_common(100), columns=['word', 'count'])
FG_top_100_words = pd.DataFrame(FG_word_counts.most_common(100), columns=['word', 'count'])
LP_top_100_words = pd.DataFrame(LP_word_counts.most_common(100), columns=['word', 'count'])
SF_top_100_words = pd.DataFrame(SF_word_counts.most_common(100), columns=['word', 'count'])

In [ ]:
top_100_words_by_parties = pd.merge(FF_top_100_words, FG_top_100_words, on = 'word', how = 'outer').merge(LP_top_100_words, on = 'word', how = 'outer').merge(SF_top_100_words, on = 'word', how = 'outer')

In [ ]:
top_100_words_by_parties.columns = ["word"] + party_names
top_100_words_by_parties.set_index("word", inplace=True)
top_100_words_by_parties = top_100_words_by_parties.fillna(0)

In [ ]:
top_100_words_by_parties_ratios = top_100_words_by_parties.div(top_100_words_by_parties.sum(axis=0), axis=1)

In [ ]:
top_100_wbp_plot = top_100_words_by_parties_ratios.loc[counts.word.to_list()].reset_index().melt(id_vars="word", value_vars=party_names)

Porównanie jaką część z sumy 100 najczęściej używanych słów przez każdą z 4 głównych partii stanowi 30 najpopularniejszych słów ogółem również dostarcza nam ciekawych informacji: 
- politcy Fianna Fail stosunkowo rzadziej mówią o ministrach i nie używają słowa 'ask', co jest powiązane z tym, że to politycy partii rządzącej przez większość analizowanego zakresu czasowego, a to politycy opozycyjni częściej zwracają się do ministrów rządu,
- politycy Fianna Fail i Sinn Fein zdecydowanie rzadziej używają słowa 'matter' od pozostałych partii, 
- słowo 'statement' nie znalazło się w ogóle wśród 100 najpopularniejszych słów dla partii Fianna Fail,
- politycy Sinn Fein zdecydowanie częściej używają określenia ogółu - 'people' aniżeli słowa 'person'; dla innych partii stosunek ten jest porównywalny,
- tematem szkół zdecydowanie częściej zajmowali się w swoich przemówieniach politycy FF i The Labour Party,
- zauważalne jest podobieństwo między częstością występowania danych słów (poruszania pewnych tematów?) przez polityków Fine Gael i The Labour Party, które wchodzą ze sobą w koalicje. 

In [ ]:
import seaborn as sns
sns.catplot(
    data=top_100_wbp_plot, kind="bar",
    y="word", x="value", hue="variable",
    palette="dark", alpha=.6, height=12, orientation="horizontal"
)
plt.show()

In [ ]:
top_100_words_by_parties_ratios['sum'] = top_100_words_by_parties_ratios.sum(axis=1)

Okazuje się, że partia Fianna Fail stosunkowo częściej od innych mówi chociażby o UE (przypomnijmy, że jest przeciwna silnej integracji). W ich retoryce pojawia się też częściej zdrowie.

Fine Gael często mówi o Gardzie - irlandzkiej policji. Pojawienie się słowa 'Affairs' może być związane z Department of Foreign Affairs. 

Sinn Fein mówi częściej o podatkach, gospodarce, bankach, pieniądzach, ale również o wspólnocie (przypomnijmy, że dążą do zjednoczenia z Irlandią Północną).

Partia Pracy odróżnia się natomiast tym, że mówi o edukacji, sprawach socjalnych i zasiłkach.


In [ ]:
party_specific_words = dict(zip(party_names, [[] for i in range(4)]))
for party in party_names:
  for word, row in top_100_words_by_parties_ratios.iterrows():
    if row[party] >= row['sum'] - row[party]:
      party_specific_words[party].append(word)
party_specific_words


Analiza bigramów i trigramów, które najczęściej pojawiają się w tekstach dostarcza kolejnych informacji o tym, co jest poruszane w przemówieniach parlamentarzystów. W tym przypadku bardziej widać zakresy tematyczne niż analizując pojedyncze słowa. Widzimy np. że ważnym tematem jest irlandzka policja, ale bardzo często pojawia się też temat zasiłków społecznych, a nawet praw człowieka. 

In [ ]:
bigrams = docs.apply(lambda doc: [span.text for span in textacy.extract.ngrams(doc, n=(2,3), min_freq=2)])

In [ ]:
bitrigrams_counts = Counter(bigrams.sum())

In [ ]:
top_30_bitrigrams = pd.DataFrame(bitrigrams_counts.most_common(30), columns=['span', 'count'])

In [ ]:
fig = px.bar(top_30_bitrigrams,orientation='h', y='span', x='count')
fig['layout']['yaxis']['autorange'] = "reversed"
fig.update_layout(bargap=0.2, font={'size':10})
img_bytes = fig.to_image(format="png", width=800, height=600, scale=2, engine='kaleido')
Image(img_bytes)

## Analiza najważniejszych terminów w przemówieniach przedstawicieli danych ministerstw

In [ ]:
ministers_df.head()

In [ ]:
ministers_df["position"].unique()

In [ ]:
ministers_df["department"].unique()

Usuwamy wiersze dotyczące premierów, wicepremierów i sekretarzy oraz zbędne kolumny. Zostawimy też dane o ministrach tylko od 6 czerwca 2002 roku tak jak w ramce danych dotyczącej przemówień.

In [ ]:
ministers_df_prepoc = ministers_df.loc[(ministers_df["department"] != "Taoiseach") & (ministers_df["department"] != "Tánaiste"),
                                       ["position", "department", "name", "memberID", "start_date", "end_date"]]
ministers_df_prepoc.start_date = pd.to_datetime(ministers_df_prepoc.start_date)
ministers_df_prepoc.end_date = pd.to_datetime(ministers_df_prepoc.end_date)

In [ ]:
ministers_df_prepoc.info()

In [ ]:
pd.isnull(ministers_df_prepoc["end_date"])

In [ ]:
ministers_df_prepoc.loc[pd.isnull(ministers_df_prepoc["end_date"])].start_date.unique()

W ramce są braki danych. Nie ma wpisanej daty zakończenia piastowania stanowiska, jeżeli minister zaczął pełnić rolę 9 marca 2011. Będziemy zatem rozważali okres do 2011-03-09.

In [ ]:
ministers_df_prepoc.loc[ministers_df_prepoc["start_date"] != np.datetime64('2011-03-09')].info()

In [ ]:
ministers_df_prepoc = ministers_df_prepoc.loc[(ministers_df_prepoc["end_date"] != np.datetime64('2011-03-09')) & (ministers_df_prepoc["end_date"] >= np.datetime64('2002-06-06'))]

Wyciągnijmy informacje o przemówieniach, które były prowadzone przez ministrów.

In [ ]:
df4 = df3.copy()
df4["department"] = "False"

In [ ]:
for index, row in df3.iterrows():
    member_id = row["memberID"]
    speech_date = row["date"]
    temp_df = ministers_df_prepoc.loc[(ministers_df_prepoc["memberID"] == member_id) &\
                                      (ministers_df_prepoc["start_date"] <= speech_date) &\
                                      (ministers_df_prepoc["end_date"] >= speech_date)]
    if(len(temp_df)):
      df4.at[index, "department"] = temp_df["department"].iloc[0]
df4.department.value_counts()

In [ ]:
df4 = df4.loc[df4["department"] != "False"]

Dla każdego przemówienia będziemy wyciągać 3 najważniejsze terminy. Następnie będziemy je zliczać, aby stwierdzić, które występują w największej liczbie przemówień.

In [ ]:
department_keywords = {dep : np.array([]) for dep in df4.department.unique()}

In [ ]:
for index, row in df4.iterrows():
    department_keywords[row["department"]] = np.append(department_keywords[row["department"]], [term[0] for term in textacy.extract.keyterms.textrank(docs[index])[0:3]])

In [ ]:
df_keywords = pd.DataFrame(columns=['depatament', 'most_frequent', 'second_most_frequent', 'third_most_frquent'])
for key in department_keywords:
  unique, counts = np.unique(department_keywords[key], return_counts=True)
  count_sort_idx = np.argsort(-counts)
  df_keywords = df_keywords.append({'depatament' : key, 'most_frequent' : unique[count_sort_idx][0],'second_most_frequent' : unique[count_sort_idx][1],'third_most_frquent' : unique[count_sort_idx][2]}, ignore_index=True)

In [ ]:
df_keywords


Przykładowe wnioski:
 - Ministerstwo Agrokultury i Żywności zajmowało się 'Single Payment Scheme'. Jest to wsparcie finansowe wypłacane rolnikom. Otrzymują oni dopłatę za hektar ziemi wykorzystywanej pod uprawę.
 - Ministerstwo Edukacji zajmowało się głównie problemami szkół podstawowych.
 - Ministerstwo Sprawiedliwości i Równości częstwo odwoływało się do pozycji Komisarza ds. Wniosków Uchodźców.
 - Ministerstwo zdrowia poruszało temat usług socjalnych. Często odwoływało się do roli Kierownika służby zdrowia
 - Ministerstwo spraw zewnętrznych często wspominało o członkostwie w Unii Europejskiej.
 - Ministerstwo środowiska przemawiało na temat programu 'Water Services Investment Programme', czyli planu inwestycji w szeroko pojętą infrastrukturę wodną.
